In [1]:
from pyvis import network as net
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib import rcParams, cycler
from matplotlib.lines import Line2D
import matplotlib.cm as cmx

In [2]:
all_df = pd.read_csv("Phase5_Data.csv")

#Categories needed for each person
#Submitter, medium, mood after conversation, extroversion scale, category of person

temp_df = all_df[['Submitter', 'Medium of Conversation', 'Mood after the conversation', 
                  'Extroversion Scale', 'General Category of Person']]

jessie_df = temp_df.query('Submitter == "Jessie"')
shaina_df = temp_df.query('Submitter == "SR"')
erin_df = temp_df.query('Submitter == "Erin"')
luke_df = temp_df.query('Submitter == "Luke"')

#display(jessie_df)
#jessie_df = jessie_df.head(5)

In [3]:
categories={
    'Friend': '#4BC6C7', #Blue
    'Stranger': '#a39c9b', #Gray
    'Work': '#FFC300', #Yellow
    'School': '#EE3110',#Red
    'Family': '#80CD0A', #Green
    'Other': '#7511D8' #Purple   
}

node_sizes = {
    1: 10,
    2: 20,
    3: 30,
    4: 40,
    5: 50,
    6: 60,
    7: 70,
    8: 80,
    9: 90,
    10: 100
}

def createSettings():
    return net.Network(notebook=True, height='800px', width='950px')

def createGraph(submitter, submitter_df):
    #Add central name node
    g = nx.Graph()
    g.add_node(-1, label=submitter, size=8, title="central node", color='black',group="central")
    #g.repulsion(node_distance=50, spring_length=200)
    msg = "placeholder"
    for index, row, in submitter_df.iterrows():
        personCategory = row['General Category of Person']
        msg = "<p><b>Category</b>: " + personCategory +"</p>"
        g.add_node(index, label=row['Medium of Conversation'], 
                   size=node_sizes[int(row['Mood after the conversation'])],
                   title=msg,
                   group="datapoint",
                   value=node_sizes[int(row['Mood after the conversation'])]*10,
                   color=categories[personCategory],
                   mass=int(row['Mood after the conversation'])
                  )
        
        g.add_edge(-1, index)
    return g




In [4]:
g1 = createSettings()
nxg = createGraph("Jessie", jessie_df)
g1.from_nx(nxg)
g1.show('jessie.html')

In [5]:
g2 = createSettings()
nxg = createGraph("Shaina", shaina_df)
g2.from_nx(nxg)
g2.show('shaina.html')

In [6]:
g3 = createSettings()
nxg = createGraph("Erin", erin_df)
g3.from_nx(nxg)
g3.show('erin.html')

In [7]:
g4 = createSettings()
nxg = createGraph("Luke", luke_df)
g4.from_nx(nxg)
g4.show('luke.html')